In [441]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
import glob
import os

In [513]:
DUMP_ID = 640

In [514]:
PATH_PREFIX = "/mnt/storage/data/vision/imagenet/training_dumps/" + str(DUMP_ID).zfill(8)

### Parameters

In [507]:
def get_values_from_files(filenames):
    arr = []
    for f in filenames:
        arr.append(np.fromfile(f, dtype=np.float32))
    return arr

In [515]:
model_param_buffer_filenames = sorted(glob.glob(PATH_PREFIX + "/model_params/*"))
gradient_buffer_filenames = sorted(glob.glob(PATH_PREFIX + "/gradients/*"))
mean_buffer_filenames = sorted(glob.glob(PATH_PREFIX + "/means/*"))
var_buffer_filenames = sorted(glob.glob(PATH_PREFIX + "/vars/*"))

In [516]:
model_params_list = get_values_from_files(model_param_buffer_filenames)
gradients_list = get_values_from_files(gradient_buffer_filenames)
means_list = get_values_from_files(mean_buffer_filenames)
vars_list = get_values_from_files(var_buffer_filenames)

In [517]:
for i in range(212, -1, -1):
    print("[" + str(min(gradients_list[i])) + "-" + str(max(gradients_list[i])) + "]")

[-0.8363113-0.013355222]
[-0.17130719-0.18393156]
[-0.07876639-0.07254439]
[0.0-0.0]
[-0.0070496295-0.0015472133]
[-0.0006786119-1.6284233e-05]
[-0.00044640462-0.00012055687]
[0.0-0.0]
[-7.1367635e-05-0.0001776251]
[-0.0011396919--0.00048585175]
[-0.0008936281--0.00043459272]
[0.0-0.0]
[7.118365e-05-0.0022217296]
[-0.14022303-0.1789879]
[-0.06851056-0.08047116]
[0.0-0.0]
[-0.0020900078-0.006729803]
[-0.0006246123-0.00012863561]
[-0.0003447849-0.0001777429]
[0.0-0.0]
[-6.000383e-05-0.00014968617]
[-0.00081462355--0.0003980487]
[-0.00059229165--0.0003164214]
[0.0-0.0]
[6.141255e-05-0.0016111501]
[-0.14089762-0.1472841]
[-0.08321755-0.08808192]
[0.0-0.0]
[-0.21473114-0.3611145]
[-0.14089762-0.1472841]
[-0.07674796-0.08845531]
[0.0-0.0]
[-0.018554522-0.02521982]
[-0.0020704411-0.0021836604]
[-0.001220143-0.0013126191]
[0.0-0.0]
[-0.0008769233-0.001178356]
[-0.001226957-1.2066989e-05]
[-0.00048195865-3.0889238e-05]
[0.0-0.0]
[-3.286407e-05-0.010720545]
[-0.29161686--0.10245483]
[-0.1101736-

### Forward Activations

In [209]:
def gather_batch_norm_data(dir_name):
    data_list = []
    for s in ["reduced", "spatial", "expanded"]:
        for f in ["means.buffer", "vars.buffer", "norm_temp.buffer", "norm.buffer"]:
            data = np.fromfile(dir_name + "/" + s + "/" + f, dtype=np.float32)
            data_list.append(data)
    return data_list

In [444]:
def gather_convblock_data(dir_name):
    data_list = []
    for f in ["reduction_applied.buffer", "reduction_activated.buffer", "spatial_applied.buffer", "spatial_activated.buffer", "expanded_applied.buffer", "combined_output.buffer", "output_activated.buffer"]:
        data = np.fromfile(dir_name + "/" + f, dtype=np.float32)
        data_list.append(data)
    return data_list

#### Initial Conv Layer and MaxPool

In [9]:
init_conv_applied = np.fromfile(PATH_PREFIX + "/activations/init_conv_applied.buffer", dtype=np.float32)
init_conv_activated = np.fromfile(PATH_PREFIX + "/activations/init_conv_activated.buffer", dtype=np.float32)
max_inds = np.fromfile(PATH_PREFIX + "/activations/max_inds.buffer", dtype=np.int32)
init_convblock_input = np.fromfile(PATH_PREFIX + "/activations/init_convblock_input.buffer", dtype=np.float32)

In [388]:
data_list = gather_batch_norm_data(PATH_PREFIX + "/activation_derivs/batch_norms/00")

In [389]:
for d in data_list:
    print("[" + str(min(d)) + " - " + str(max(d)) + "]")

[0.0 - 0.0]
[0.0 - 0.0]
[-835.5199 - 907.6788]
[0.0 - 0.0]
[0.0 - 0.0]
[0.0 - 0.0]
[-447.0688 - 381.68115]
[0.0 - 0.0]
[0.0 - 0.0]
[0.0 - 0.0]
[-136.80064 - 149.83704]
[0.0 - 0.0]


#### Final Avg Pool, Fully Connected, and Softmax

In [10]:
final_avg_pool = np.fromfile(PATH_PREFIX + "/activations/final_avg_pool.buffer", dtype=np.float32)
fc_output = np.fromfile(PATH_PREFIX + "/activations/fc_output.buffer", dtype=np.float32)
prediction = np.fromfile(PATH_PREFIX + "/activations/softmax.buffer", dtype=np.float32)
correct_classes = np.fromfile(PATH_PREFIX + "/activations/correct_classes.buffer", dtype=np.int32)

In [30]:
min(fc_output)

-4.580309

### Backward Activations

In [436]:
d_init_conv_applied = np.fromfile(PATH_PREFIX + "/activation_derivs/init_conv_applied.buffer", dtype=np.float32)
d_init_conv_activated = np.fromfile(PATH_PREFIX + "/activation_derivs/init_conv_activated.buffer", dtype=np.float32)
d_init_convblock_input = np.fromfile(PATH_PREFIX + "/activation_derivs/init_convblock_input.buffer", dtype=np.float32)
d_final_avg_pool = np.fromfile(PATH_PREFIX + "/activation_derivs/final_avg_pool.buffer", dtype=np.float32)
d_fc_output = np.fromfile(PATH_PREFIX + "/activation_derivs/fc_output.buffer", dtype=np.float32)
d_prediction = np.fromfile(PATH_PREFIX + "/activation_derivs/softmax.buffer", dtype=np.float32)

In [438]:
print("[" + str(min(d_final_avg_pool)) + " - " + str(max(d_final_avg_pool)) + "]")

[-0.00401514 - 0.004302698]


In [486]:
DUMP_ID = 0
PATH_PREFIX = "/mnt/storage/data/vision/imagenet/training_dumps/" + str(DUMP_ID).zfill(8)
conv_data_list = gather_convblock_data(PATH_PREFIX + "/activation_derivs/conv_blocks/08")

In [487]:
for c in conv_data_list:
    print("[" + str(min(c)) + " - " + str(max(c)) + "]")

[-3.4619005 - 2.903917]
[-0.17878185 - 0.17705059]
[-0.75275075 - 0.84935224]
[-0.082793474 - 0.08686427]
[-0.6543597 - 0.62088454]
[-0.025375048 - 0.028530437]
[-0.025375048 - 0.028530437]


In [488]:
DUMP_ID = 2
PATH_PREFIX = "/mnt/storage/data/vision/imagenet/training_dumps/" + str(DUMP_ID).zfill(8)
conv_data_list = gather_convblock_data(PATH_PREFIX + "/activation_derivs/conv_blocks/08")

In [489]:
for c in conv_data_list:
    print("[" + str(min(c)) + " - " + str(max(c)) + "]")

[-3.0024638 - 3.4002678]
[-0.18356527 - 0.17030282]
[-0.8715495 - 0.8251272]
[-0.1011737 - 0.08904381]
[-0.74025804 - 0.6807653]
[-0.027166499 - 0.028757486]
[-0.027166499 - 0.028757486]
